# Training an SNN using surrogate gradients! 72c128t


Train your first SNN in JAX in less than 10 minutes without needing a heavy-duty GPU!

In [1]:
# implement our SNN in DeepMind's Haiku
import haiku as hk

# JAX imports
import jax
import jmp

# rendering tools
# for surrogate loss training.
import optax
from jax import numpy as jnp
from jax_tqdm import scan_tqdm

import spyx
import spyx.nn as snn

%matplotlib notebook

In [2]:
policy = jmp.get_policy('half')


hk.mixed_precision.set_policy(hk.Linear, policy)
hk.mixed_precision.set_policy(snn.LIF, policy)
hk.mixed_precision.set_policy(snn.LI, policy)

In [3]:
shd_dl = spyx.data.SHD_loader(256,1024,256)

In [4]:
key = jax.random.PRNGKey(0)
x, y = shd_dl.train_epoch(key)

In [5]:
y.shape

(25, 256)

In [6]:
surrogate = spyx.axn.Axon(spyx.axn.arctan())

In [7]:
def snn_64(x):
    
    x = hk.BatchApply(hk.Linear(64, with_bias=False))(x)
    
    core = hk.DeepRNN([
        snn.LIF((64,), activation=surrogate),
        hk.Linear(64, with_bias=False),
        snn.LIF((64,), activation=surrogate),
        hk.Linear(20, with_bias=False),
        snn.LI((20,))
    ])
    
    # static unroll for maximum performance
    spikes, V = hk.dynamic_unroll(core, x, core.initial_state(x.shape[0]), time_major=False, unroll=16)
    
    return spikes, V

In [8]:
def snn_128(x):
    
    x = hk.BatchApply(hk.Linear(128, with_bias=False))(x)
    
    core = hk.DeepRNN([
        snn.LIF((128,), activation=surrogate),
        hk.Linear(128, with_bias=False),
        snn.LIF((128,), activation=surrogate),
        hk.Linear(20, with_bias=False),
        snn.LI((20,))
    ])
    
    # static unroll for maximum performance
    spikes, V = hk.dynamic_unroll(core, x, core.initial_state(x.shape[0]), time_major=False, unroll=16)
    
    return spikes, V

In [9]:
def snn_256(x):
    
    x = hk.BatchApply(hk.Linear(256, with_bias=False))(x)
    
    core = hk.DeepRNN([
        snn.LIF((256,), activation=surrogate),
        hk.Linear(256, with_bias=False),
        snn.LIF((256,), activation=surrogate),
        hk.Linear(20, with_bias=False),
        snn.LI((20,))
    ])
    
    # static unroll for maximum performance
    spikes, V = hk.dynamic_unroll(core, x, core.initial_state(x.shape[0]), time_major=False, unroll=16)
    
    return spikes, V

In [10]:
def snn_512(x):
    
    x = hk.BatchApply(hk.Linear(512, with_bias=False))(x)
    
    core = hk.DeepRNN([
        snn.LIF((512,), activation=surrogate),
        hk.Linear(512, with_bias=False),
        snn.LIF((512,), activation=surrogate),
        hk.Linear(20, with_bias=False),
        snn.LI((20,))
    ])
    
    # static unroll for maximum performance
    spikes, V = hk.dynamic_unroll(core, x, core.initial_state(x.shape[0]), time_major=False, unroll=16)
    
    return spikes, V

In [11]:
key = jax.random.PRNGKey(0)
# Since there's nothing stochastic about the network, we can avoid using an RNG as a param!
SNN_64 = hk.without_apply_rng(hk.transform(snn_64))
params_64 = SNN_64.init(rng=key, x=x[0])

In [12]:
key = jax.random.PRNGKey(0)
# Since there's nothing stochastic about the network, we can avoid using an RNG as a param!
SNN_128 = hk.without_apply_rng(hk.transform(snn_128))
params_128 = SNN_128.init(rng=key, x=x[0])

In [13]:
key = jax.random.PRNGKey(0)
# Since there's nothing stochastic about the network, we can avoid using an RNG as a param!
SNN_256 = hk.without_apply_rng(hk.transform(snn_256))
params_256 = SNN_256.init(rng=key, x=x[0])

In [14]:
key = jax.random.PRNGKey(0)
# Since there's nothing stochastic about the network, we can avoid using an RNG as a param!
SNN_512 = hk.without_apply_rng(hk.transform(snn_512))
params_512 = SNN_512.init(rng=key, x=x[0])

In [15]:
def gd(SNN, params, dl, epochs=300, schedule=4e-4):
    
    aug = spyx.data.shift_augment(max_shift=16) # need to make this stateless

    opt = optax.chain(
        optax.centralize(),
        optax.lion(learning_rate=schedule),
    )
    # create and initialize the optimizer
    opt_state = opt.init(params)
    grad_params = params
        
    # define and compile our eval function that computes the loss for our SNN
    @jax.jit
    def net_eval(weights, events, targets):
        readout = SNN.apply(weights, events)
        traces, V_f = readout
        return spyx.fn.integral_crossentropy(traces, targets)
        
    # Use JAX to create a function that calculates the loss and the gradient!
    surrogate_grad = jax.value_and_grad(net_eval) 
        
    rng = jax.random.PRNGKey(0)        
    
    # compile the meat of our training loop for speed
    @jax.jit
    def train_step(state, data):
        grad_params, opt_state = state
        events, targets = data # fix this
        events = jnp.unpackbits(events, axis=1) # decompress temporal axis
        # compute loss and gradient                    # need better augment rng
        loss, grads = surrogate_grad(grad_params, aug(events, jax.random.fold_in(rng,jnp.sum(targets))), targets)
        # generate updates based on the gradients and optimizer
        updates, opt_state = opt.update(grads, opt_state, grad_params)
        # return the updated parameters
        new_state = [optax.apply_updates(grad_params, updates), opt_state]
        return new_state, loss
    
    # For validation epochs, do the same as before but compute the
    # accuracy, predictions and losses (no gradients needed)
    @jax.jit
    def eval_step(grad_params, data):
        events, targets = data # fix
        events = jnp.unpackbits(events, axis=1)
        readout = SNN.apply(grad_params, events)
        traces, V_f = readout
        acc, pred = spyx.fn.integral_accuracy(traces, targets)
        loss = spyx.fn.integral_crossentropy(traces, targets)
        return grad_params, jnp.array([acc, loss])
        
    
    val_data = dl.val_epoch()
    
    # Here's the start of our training loop!
    @scan_tqdm(epochs)
    def epoch(epoch_state, epoch_num):
        curr_params, curr_opt_state = epoch_state
        
        shuffle_rng = jax.random.fold_in(rng, epoch_num)
        train_data = dl.train_epoch(shuffle_rng)
        
        # train epoch
        end_state, train_loss = jax.lax.scan(
            train_step,# func
            [curr_params, curr_opt_state],# init
            train_data,# xs
            train_data.obs.shape[0]# len
        )
        
        new_params, _ = end_state
            
        # val epoch
        _, val_metrics = jax.lax.scan(
            eval_step,# func
            new_params,# init
            val_data,# xs
            val_data.obs.shape[0]# len
        )

        
        return end_state, jnp.concatenate([jnp.expand_dims(jnp.mean(train_loss),0), jnp.mean(val_metrics, axis=0)])
    # end epoch
    
    # epoch loop
    final_state, metrics = jax.lax.scan(
        epoch,
        [grad_params, opt_state], # metric arrays
        jnp.arange(epochs), # 
        epochs # len of loop
    )
    
    final_params, _ = final_state
    
                
    # return our final, optimized network.       
    return final_params, metrics

In [16]:
def test_gd(SNN, params, dl):

    @jax.jit
    def test_step(params, data):
        events, targets = data
        events = jnp.unpackbits(events, axis=1)
        readout = SNN.apply(params, events)
        traces, V_f = readout
        acc, pred = spyx.fn.integral_accuracy(traces, targets)
        loss = spyx.fn.integral_crossentropy(traces, targets)
        return params, [acc, loss, pred, targets]
    
    test_data = dl.test_epoch()
    
    _, test_metrics = jax.lax.scan(
            test_step,# func
            params,# init
            test_data,# xs
            test_data.obs.shape[0]# len
    )
    
    acc = jnp.mean(test_metrics[0])
    loss = jnp.mean(test_metrics[1])
    preds = jnp.array(test_metrics[2]).flatten()
    tgts = jnp.array(test_metrics[3]).flatten()
    return acc, loss, preds, tgts

In [17]:
from time import time

In [18]:
schedule = 2e-4

start = time()
grad_params_64, metrics_64 = gd(SNN_64, params_64, shd_dl, epochs=500, schedule=schedule) # :36 seconds for loop.
total = time() - start
print(total, "seconds elapsed")

  0%|          | 0/500 [00:00<?, ?it/s]

303.4728798866272 seconds elapsed


In [19]:
print("Performance: train_loss={}, val_acc={}, val_loss={}".format(*metrics_64[-1]))

Performance: train_loss=1.9385422468185425, val_acc=0.8287760615348816, val_loss=1.9123954772949219


In [20]:
schedule = 1e-4

start = time()
grad_params_128, metrics_128 = gd(SNN_128, params_128, shd_dl, epochs=500, schedule=schedule) # :42 seconds for loop
total = time() - start
print(total, "seconds elapsed")

  0%|          | 0/500 [00:00<?, ?it/s]

347.22015047073364 seconds elapsed


In [21]:
print("Performance: train_loss={}, val_acc={}, val_loss={}".format(*metrics_128[-1]))

Performance: train_loss=1.8767774105072021, val_acc=0.8736979365348816, val_loss=1.8424882888793945


In [22]:
schedule = 1e-4

start = time()
grad_params_256, metrics_256 = gd(SNN_256, params_256, shd_dl, epochs=500, schedule=schedule) # :54 seconds for loop
total = time() - start
print(total, "seconds elapsed")

  0%|          | 0/500 [00:00<?, ?it/s]

443.76652693748474 seconds elapsed


In [23]:
print("Performance: train_loss={}, val_acc={}, val_loss={}".format(*metrics_256[-1]))

Performance: train_loss=1.8004436492919922, val_acc=0.9036458730697632, val_loss=1.7923839092254639


In [24]:
schedule = .5e-4

start = time()
grad_params_512, metrics_512 = gd(SNN_512, params_512, shd_dl, epochs=500, schedule=schedule) # :42 seconds for loop
total = time() - start
print(total, "seconds elapsed")

  0%|          | 0/500 [00:00<?, ?it/s]

697.9654779434204 seconds elapsed


In [25]:
print("Performance: train_loss={}, val_acc={}, val_loss={}".format(*metrics_512[-1]))

Performance: train_loss=1.797947645187378, val_acc=0.8951823115348816, val_loss=1.7962278127670288


## Evaluation Time

Now we'll run the network on the test set and see what happens:

In [26]:
acc, loss, preds, tgts = test_gd(SNN_512, grad_params_512, shd_dl)
print("Accuracy:", acc, "Loss:", loss)

Accuracy: 0.75 Loss: 2.0277839
